In [7]:
from openai import OpenAI
import base64
from io import BytesIO
import requests

In [8]:
# Original local endpoints (commented out)
openai_api_key = "EMPTY"
openai_api_base_list = [
   "http://localhost:8000/v1",
   # "http://44.247.43.195:18901/v1",
   # os.environ.get("LLM_AS_A_JUDGE_BASE", "http://10.39.3.123:18901/v1"),
]

client_list = []
for api_base in openai_api_base_list:
    client = OpenAI(
       api_key=openai_api_key,
       base_url=api_base,
    )
    client_list.append(client)
    model_name_list = []
for client in client_list:
    response = requests.get(f"{api_base}/models")
    models = response.json()
    model_name_list.append(models['data'][0]['id'])

In [9]:
client

In [11]:
from vl_agent import (
    rubrics_judge,
    extract_answer,
    RUBRIC_VS_USAGE,
    RUBRICS_JUDGE_SYSTEM_PROMPT_VS
)

SyntaxError: unterminated string literal (detected at line 336) (vl_agent.py, line 336)

In [ ]:
# Image path
IMAGE_PATH = "/workspace/shared/datasets/CodeV_images/45222.jpg"


def test_vs_score_1_0():
    """VS: Perfect score - proper cropping and color analysis."""
    print("\n" + "=" * 80)
    print("TEST 1: VS - Score 1.0 (Proper cropping + analysis)")
    print("=" * 80)

    question = "what's the color of vase"
    ground_truth = "blue"

    response = """<think>I need to locate the vase in the image and analyze its color. Let me crop the vase region to get a better view.</think>
<code>
from PIL import Image
import numpy as np

img = Image.open('45222.jpg')
img_array = np.array(img)

# Crop vase region (estimated coordinates)
vase_crop = img.crop((200, 200, 500, 400))
vase_crop.save('vase_region.jpg')

# Analyze dominant color in cropped region
vase_array = np.array(vase_crop)
avg_color = vase_array.mean(axis=(0,1))
print(f"Average RGB: {avg_color}")

# Blue channel is dominant
if avg_color[2] > avg_color[0] and avg_color[2] > avg_color[1]:
    color = "blue"
print(f"Vase color: {color}")
</code>
<sandbox_output>Average RGB: [45.2 78.3 156.8]
Vase color: blue</sandbox_output>
<answer>blue</answer>"""

    extracted = extract_answer(response)

    # Create mock PIL images
    try:
        from PIL import Image
        img = Image.open(IMAGE_PATH)
        mock_images = [img.crop((200, 200, 500, 400))]
    except:
        mock_images = None

    score = rubrics_judge(response, question, ground_truth, RUBRIC_VS_USAGE,
                          extracted, RUBRICS_JUDGE_SYSTEM_PROMPT_VS, mock_images, IMAGE_PATH)